In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
df= pd.read_csv(r"D:\data_analytics\projects\chat gpt vs deepseek\deepseek_vs_chatgpt.csv")

df_raw = df.copy()
df["Session_Duration_sec"].describe()


count    10000.000000
mean        28.533700
std         14.090348
min          5.000000
25%         17.000000
50%         27.000000
75%         38.000000
max         60.000000
Name: Session_Duration_sec, dtype: float64

In [4]:
df.shape

(10000, 28)

In [3]:
df.columns

Index(['Date', 'Month_Num', 'Weekday', 'AI_Platform', 'AI_Model_Version',
       'Active_Users', 'New_Users', 'Churned_Users', 'Daily_Churn_Rate',
       'Retention_Rate', 'User_ID', 'Query_Type', 'Input_Text',
       'Input_Text_Length', 'Response_Tokens', 'Topic_Category', 'User_Rating',
       'User_Experience_Score', 'Session_Duration_sec', 'Device_Type',
       'Language', 'Response_Accuracy', 'Response_Speed_sec',
       'Response_Time_Category', 'Correction_Needed', 'User_Return_Frequency',
       'Customer_Support_Interactions', 'Region'],
      dtype='object')

In [5]:
df.isna().sum().sort_values(ascending = False)

Response_Accuracy                379
Date                               0
Weekday                            0
Month_Num                          0
AI_Model_Version                   0
Active_Users                       0
New_Users                          0
Churned_Users                      0
Daily_Churn_Rate                   0
Retention_Rate                     0
User_ID                            0
AI_Platform                        0
Query_Type                         0
Input_Text                         0
Response_Tokens                    0
Input_Text_Length                  0
User_Rating                        0
User_Experience_Score              0
Session_Duration_sec               0
Topic_Category                     0
Device_Type                        0
Language                           0
Response_Speed_sec                 0
Response_Time_Category             0
Correction_Needed                  0
User_Return_Frequency              0
Customer_Support_Interactions      0
R

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Date                           10000 non-null  object 
 1   Month_Num                      10000 non-null  int64  
 2   Weekday                        10000 non-null  object 
 3   AI_Platform                    10000 non-null  object 
 4   AI_Model_Version               10000 non-null  object 
 5   Active_Users                   10000 non-null  int64  
 6   New_Users                      10000 non-null  int64  
 7   Churned_Users                  10000 non-null  int64  
 8   Daily_Churn_Rate               10000 non-null  float64
 9   Retention_Rate                 10000 non-null  float64
 10  User_ID                        10000 non-null  object 
 11  Query_Type                     10000 non-null  object 
 12  Input_Text                     10000 non-null  

In [7]:
# standardize column names 
df.columns = df.columns.str.lower().str.strip()

In [8]:
# convert date
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df["date"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: date
Non-Null Count  Dtype         
--------------  -----         
10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 78.3 KB


In [9]:
# drop obviously broken rows
df = df.dropna(subset=['date', 'ai_platform'])      # drop only those rows where date and ai_platform in missing

In [10]:
# Fixing Datatypes

int_cols = [
    'active_users','new_users','churned_users',
    'response_tokens','input_text_length',
    'session_duration_sec','customer_support_interactions'
]

float_cols = [
    'daily_churn_rate','retention_rate',
    'user_rating','user_experience_score',
    'response_accuracy','response_speed_sec'
]

df[int_cols] = df[int_cols].apply(pd.to_numeric, errors='coerce')
df[float_cols] = df[float_cols].apply(pd.to_numeric, errors='coerce')

In [11]:
df["date"].dt.year.value_counts()  # 3 years of dataset

date
2024    6607
2023    2873
2025     520
Name: count, dtype: int64

In [12]:
df = df.drop(columns=['input_text'])   # not required for the analysis

In [13]:
# Feature Engineering

df["platform"] = df["ai_platform"].str.lower() 
df["platform"].sample(5)
df["platform"].value_counts()

platform
chatgpt     5076
deepseek    4924
Name: count, dtype: int64

In [14]:
# Engagement Bucket

df["session_duration_sec"].describe()   # the time stamps are not in seconds so we will do that

# this is how it looked like before the conversion:

# mean        28.533700
# std         14.090348
# min          5.000000
# 25%         17.000000
# 50%         27.000000
# 75%         38.000000
# max         60.000000

count    10000.000000
mean        28.533700
std         14.090348
min          5.000000
25%         17.000000
50%         27.000000
75%         38.000000
max         60.000000
Name: session_duration_sec, dtype: float64

In [ ]:
# df["session_duration_sec"] = df["session_duration_sec"] * 60    # converted to seconds (only_once)
df["session_duration_sec"].describe()

df["session_duration_sec"].sample(10)

23      1920
8015     720
8235    2940
7575    1440
1829    1560
139     2340
7197    3420
9585    2280
2324    2340
7054    2580
Name: session_duration_sec, dtype: int64

In [16]:
df['engagement_level'] = pd.cut(
    df['session_duration_sec'],
    bins=[0, 300, 900, 1800, 99999],
    labels=['Very Low','Low','Medium','High'],
    include_lowest=True
)
df["engagement_level"].value_counts()



engagement_level
High        4220
Medium      3676
Low         1970
Very Low     134
Name: count, dtype: int64

In [17]:
# the result looks like this: High > Medium > Low > Very Low
# this arrises the question that is the engagemnet real, or is it session definition bias?

# some system defines "session" as the time between the first an dthe last interaction, if user leave sthe tab open the session extends.

# to check the real engagemnet we are finding the corelation between the "session time" and "response tocken"
# if 'session_duration_sec' and 'response_tokens' both are high, thats the real engagemnet 

df[['session_duration_sec', 'response_tokens']].corr()

# and it shows that the users are not actively interacting with the AI for most of that time.

,session_duration_sec,response_tokens
session_duration_sec,1.0000,0.0099
response_tokens,0.0099,1.0000


In [18]:
df["session_duration_sec"].describe()

count    10000.000000
mean      1712.022000
std        845.420904
min        300.000000
25%       1020.000000
50%       1620.000000
75%       2280.000000
max       3600.000000
Name: session_duration_sec, dtype: float64

In [19]:
df['response_tokens'].describe() 

# the maximum session time is 60 minutes and the maximum response tocken is 500 which seems a little off because

# In real usage, some people might spend more than an hour in a session or receive very long responses.
# In this dataset, those cases are grouped at the maximum value instead of being recorded precisely, as in this data the session has stopped exactly at 60 min
# So the data is good for comparison and trend analysis, but not for measuring extreme behavior.

count    10000.000000
mean       274.765100
std        130.077225
min         50.000000
25%        162.000000
50%        276.000000
75%        386.250000
max        500.000000
Name: response_tokens, dtype: float64

In [20]:

df['session_length_category'] = pd.cut(
    df['session_duration_sec'],
    bins=[0, 300, 900, 1800, 99999],
    labels=['Short','Medium','Long','Very Long'],
    include_lowest=True
)
df['session_length_category'].value_counts()

session_length_category
Very Long    4220
Long         3676
Medium       1970
Short         134
Name: count, dtype: int64

In [21]:
df['token_engagement'] = pd.cut(
    df['response_tokens'],
    bins=[0, 150, 300, 400, 500],
    labels=['Low','Moderate','High','Very High'],
    include_lowest=True
)
df['token_engagement'].value_counts()

token_engagement
Moderate     3277
High         2280
Low          2257
Very High    2186
Name: count, dtype: int64

In [ ]:
# OVERALL AI PERFORMANCE AND USAGE ANALYSIS

In [46]:
# AI impact by user background                       

df.head().T

ai_impact = df.groupby("topic_category").agg({
    "user_rating" : "mean",
    "response_accuracy" : "mean",
    "response_accuracy" : "mean",
    "correction_needed" : "mean"
    }).sort_values("user_rating", ascending = False)

ai_impact["user_rating"] = ai_impact["user_rating"].round(1)
ai_impact["response_accuracy"] = (ai_impact["response_accuracy"]*100).round(1).map(lambda x: f"{x}%")
ai_impact["correction_needed"] = (ai_impact["correction_needed"]*100).round(1).map(lambda x: f"{x}%")

ai_impact

# (THE LAMBDA PART HERE IS CONVERTING THE NUMBERS IN STRING, WHILE DOING THE ANALYSIS WE SHOULD CONVERT IT BACK TO NUMERIC)

# AI performs significantly better on technical problem-solving tasks such as algorithm explanation, debugging, and code optimization, 
# where user ratings average around 4.8 and response accuracy is close to 90%. In contrast, content-oriented and advisory topics like content 
# creation, professional writing, and best practices show lower user satisfaction (around 4.0) and reduced accuracy (~80%), indicating that 
# AI currently handles structured, objective tasks more effectively than subjective or guideline-based queries

,user_rating,response_accuracy,correction_needed
topic_category,,,
Algorithm Explanation,4.8,89.9%,13.7%
Code Optimization,4.8,90.0%,14.1%
Debugging,4.8,90.0%,13.7%
Implementation Help,4.8,89.9%,15.9%
Content Creation,4.0,80.3%,15.6%
Best Practices,4.0,80.0%,14.9%
Education,4.0,80.3%,14.2%
Professional Writing,4.0,80.5%,13.6%


In [ ]:
# Language & region impact

language_region_impact = df.groupby("language").agg({
    "response_accuracy" : "mean",
    "user_rating" : "mean"
}).sort_values("response_accuracy", ascending = False)

language_region_impact["user_rating"] = language_region_impact["user_rating"].round(1)
language_region_impact["response_accuracy"] = (language_region_impact["response_accuracy"]*100).round(1).map(lambda x: f"{x}%" )

# changing the language column in proper language
#==============================================================================
language_map = {
    'en': 'English',
    'es': 'Spanish',
    'fr': 'French',
    'de': 'German',
    'zh': 'Chinese'
}

df["language"] = df["language"].map(language_map).fillna(df["language"])
# ==============================================================================
language_region_impact

# The AI works equally well across languages. English does not have a noticeable advantage — accuracy stays almost the same regardless of language.
# What matters more is how clear and structured the question is, not the language used.

,response_accuracy,user_rating
language,,
German,85.3%,4.4
French,85.1%,4.4
Spanish,85.1%,4.4
English,84.9%,4.4
Chinese,84.8%,4.4


In [ ]:
# Device-based help (mobile vs desktop)

Device_based  = df.groupby("device_type").agg({
    "session_duration_sec" : "mean",
    "response_tokens" : "mean",
    "user_rating" : "mean"
})

Device_based["user_rating"] = Device_based["user_rating"].round(2)
Device_based["session_duration_sec"] = (Device_based["session_duration_sec"]/60).round(1).astype(str) + " min"
Device_based["response_tokens"] = Device_based["response_tokens"].astype(int)

Device_based

# Session duration remains consistently around 28–29 minutes across all device types, indicating that user engagement with AI is stable 
# regardless of access medium. Response length and user ratings also show minimal variation, suggesting a device-agnostic user experience.

,session_duration_sec,response_tokens,user_rating
device_type,,,
Laptop/Desktop,28.3 min,273,4.38
Mobile,28.6 min,276,4.40
Smart Speaker,28.1 min,265,4.36
Tablet,29.1 min,272,4.41


In [69]:
# Major performance comparison: DeepSeek vs ChatGPT

In [92]:
# core comparision metrics

platform_performance = df.groupby("ai_platform").agg({
    "user_rating" : "mean",
    "response_speed_sec" : "mean",
    "retention_rate" : "mean",
    "response_accuracy" : "mean",
    "correction_needed" : "mean",
})

platform_performance["user_rating"] = platform_performance["user_rating"].round(2)
platform_performance["response_speed_sec"] = platform_performance["response_speed_sec"].round(1).astype(str) + " sec"
platform_performance["response_accuracy"] = (platform_performance["response_accuracy"]*100).round(2).astype(str) + "%"
platform_performance["retention_rate"] = (platform_performance["retention_rate"]*100).astype(str) + "%"
platform_performance["correction_needed"] = (platform_performance["correction_needed"]*100).round(2).astype(str) + "%"

platform_performance

# DeepSeek demonstrates superior performance in speed, accuracy, and user satisfaction, indicating a stronger response quality experience.
# However, identical retention rates suggest that platform loyalty is influenced by factors beyond raw performance, such as familiarity, 
# or use-case dependency.


,user_rating,response_speed_sec,retention_rate,response_accuracy,correction_needed
ai_platform,,,,,
ChatGPT,4.0,3.4 sec,95.0%,80.26%,14.58%
DeepSeek,4.8,1.2 sec,95.0%,89.97%,14.34%


In [ ]:
# Platform × Topic matrix

query_performance = df.groupby(['ai_platform','query_type']).agg({
    'user_rating':'mean',
    'correction_needed':'mean'
}).reset_index()

query_performance["user_rating"] = query_performance["user_rating"].round(2)
query_performance["correction_needed"] = (query_performance["correction_needed"]*100).round(2).astype(str) + "%"

query_performance

# DeepSeek performs best on technical queries, where users rate it highly and also need fewer corrections. This shows that its answers are not 
# only liked, but are also more accurate and usable.
# For general queries, users still rate DeepSeek highly, but they make more edits. This suggests that people enjoy the responses, but often tweak 
# them for tone, clarity, or personal preference rather than fixing mistakes.

# Technical query data for ChatGPT was not present in the dataset, limiting direct cross-platform comparison for this category.

,ai_platform,query_type,user_rating,correction_needed
0,ChatGPT,General,4.0,14.58%
1,DeepSeek,General,4.8,15.89%
2,DeepSeek,Technical,4.8,13.81%


In [ ]:
final_df = df[[
    # Time
    'date', 'weekday', 'month_num',

    # Platform & model
    'ai_platform', 'ai_model_version',

    # User metrics
    'active_users', 'new_users', 'churned_users',
    'daily_churn_rate', 'retention_rate',

    # Engagement
    'session_duration_sec',
    'session_length_category',
    'response_tokens',
    'token_engagement',

    # Performance
    'response_speed_sec',
    'response_accuracy',
    'response_time_category',
    'correction_needed',

    # Experience
    'user_rating',
    'user_experience_score',
    'user_return_frequency',

    # Context
    'query_type',
    'topic_category',
    'device_type',
    'language',
    'region'
]]

#final_df.to_csv("ai_platform_sessions_flat.csv", index=False)